## Experminet Notebook
This notebook includes the process of training multiple models, monitoring performance, comparison and model selection over the same dataset.

## Global Variables

## Initiliazation

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LassoCV
from sklearn.linear_model import RidgeCV, LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

from datetime import datetime

In [2]:
dataframe = pd.read_csv("./KAG_energydata_complete.csv")
dataframe

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.890000,47.596667,19.200000,44.790000,19.790000,44.730000,19.000000,...,17.033333,45.5300,6.600000,733.5,92.000000,7.000000,63.000000,5.300000,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.890000,46.693333,19.200000,44.722500,19.790000,44.790000,19.000000,...,17.066667,45.5600,6.483333,733.6,92.000000,6.666667,59.166667,5.200000,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.890000,46.300000,19.200000,44.626667,19.790000,44.933333,18.926667,...,17.000000,45.5000,6.366667,733.7,92.000000,6.333333,55.333333,5.100000,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.890000,46.066667,19.200000,44.590000,19.790000,45.000000,18.890000,...,17.000000,45.4000,6.250000,733.8,92.000000,6.000000,51.500000,5.000000,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.890000,46.333333,19.200000,44.530000,19.790000,45.000000,18.890000,...,17.000000,45.4000,6.133333,733.9,92.000000,5.666667,47.666667,4.900000,10.084097,10.084097
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19730,2016-05-27 17:20:00,100,0,25.566667,46.560000,25.890000,42.025714,27.200000,41.163333,24.700000,...,23.200000,46.7900,22.733333,755.2,55.666667,3.333333,23.666667,13.333333,43.096812,43.096812
19731,2016-05-27 17:30:00,90,0,25.500000,46.500000,25.754000,42.080000,27.133333,41.223333,24.700000,...,23.200000,46.7900,22.600000,755.2,56.000000,3.500000,24.500000,13.300000,49.282940,49.282940
19732,2016-05-27 17:40:00,270,10,25.500000,46.596667,25.628571,42.768571,27.050000,41.690000,24.700000,...,23.200000,46.7900,22.466667,755.2,56.333333,3.666667,25.333333,13.266667,29.199117,29.199117
19733,2016-05-27 17:50:00,420,10,25.500000,46.990000,25.414000,43.036000,26.890000,41.290000,24.700000,...,23.200000,46.8175,22.333333,755.2,56.666667,3.833333,26.166667,13.233333,6.322784,6.322784


## Research Engine

A class to train models with different specifications in just one line.

In [3]:
class ModelCreator:
  def __init__(self, weekday_enc='ohe', hour_enc='ohe', reg_type='lasso', poly_deg=1, random_state=None, trainer=None, split_rate=0.2):
    self.params = {
      'weekday_enc': weekday_enc,
      'hour_enc': hour_enc,
      'reg_type': reg_type,
      'poly_deg': poly_deg,
      'random_state': random_state,
      'trainer': trainer,
      'split_rate': split_rate
    }
    # self.model = None
    # self.results = {}
    self.encoders = {
      'hour': None,
      'weekday': None
    }
    self.ohe_counts = None
    self.poly = None
    
  def __cyclic_encode(self, data, max_val):
    return np.column_stack([
        np.sin(2 * np.pi * data / max_val),
        np.cos(2 * np.pi * data / max_val)
    ])

  def __ohe_encode(self, data, key):
    if self.encoders[key] == None:
      encoder_data = OneHotEncoder(sparse_output=False)
      self.encoders[key] = encoder_data
      encoder_data.fit(data.reshape(-1, 1))
      # return (encoder_data.fit_transform(data.reshape(-1, 1)), encoder_data)
    return (self.encoders[key].transform(data.reshape(-1, 1)), self.encoders[key])
      

  def __binned_encode(self, data):
    pass
    
  def _process_dataframe(self, df):
    # --- Handle The Dataframe ---
    dataframe = df.copy()
    date_column = pd.to_datetime(dataframe['date'])
    hours = date_column.dt.hour.values # extracts hour column as [0, 0, ..., 2, 2, ...]
    weekdays = date_column.dt.weekday.values # extracting weekdays like [0, 0, ..., 1, 1, ...]
    dataframe = dataframe.drop('date', axis=1)
    
    # --- Handle Hour Encoding ---
    if self.params['hour_enc'] == 'ohe':
      hour_encoded, encoder_hour = self.__ohe_encode(hours, 'hour')
      hour_column_names = encoder_hour.get_feature_names_out(['hour'])
      self.encoder_hour = encoder_hour
    elif self.params['hour_enc'] == 'trig':
      hour_encoded = self.__cyclic_encode(hours, 24)
      hour_column_names = np.array(['hour_sin', 'hour_cos'])
    elif self.params['hour_enc'] == 'binned':
      pass
    else: raise Exception("this hour_enc is not supported or is not written correctly, please double check. supported hour_enc values: ohe, trig, binned")
    # put the hour-of-day into dataframe
    hour_encoded_df = pd.DataFrame(hour_encoded, columns=hour_column_names)
    dataframe = pd.concat([hour_encoded_df, dataframe], axis=1)
    
    # --- Handle Weekday Encoding ---   
    if self.params['weekday_enc'] == 'ohe':
      weekdays_encoded, encoder_weekdays = self.__ohe_encode(weekdays, 'weekday')
      weekdays_column_names = encoder_weekdays.get_feature_names_out(['weekday'])
    elif self.params['weekday_enc'] == 'trig':
      weekdays_encoded = self.__cyclic_encode(weekdays, 7)
      weekdays_column_names = np.array(['weekday_sin', 'weekday_cos'])
    elif self.params['weekday_enc'] == 'binned':
      pass
    # Putting weekday columns into dataframe
    week_encoded_df = pd.DataFrame(weekdays_encoded, columns=weekdays_column_names)
    dataframe = pd.concat([week_encoded_df, dataframe], axis=1)
    
    # --- Handle Polynomial Features ---
    if 'Appliances' in df:
      X = dataframe.drop('Appliances', axis=1)
      y = dataframe[['Appliances']]
    else:
      X = dataframe
      y = None
    
    if self.params['poly_deg'] > 1:
      if self.poly == None:
        poly = PolynomialFeatures(degree=self.params['poly_deg'], include_bias=False)
        X = poly.fit_transform(X)
        self.poly = poly
      else:
        poly = self.poly
        X = poly.transform(X)
      # Filtering out bad ohe combinations
      if self.ohe_counts == None:
        ohe_counts = []
        if self.params['weekday_enc'] == 'ohe':
          if self.params['hour_enc'] == 'ohe':
            ohe_counts = [(0, 7), (7, 7 + 24)]
          elif self.params['hour_enc'] == 'binned':
            pass
          elif self.params['hour_enc'] == 'trig':
            ohe_counts = [(0, 7)]
        if self.params['weekday_enc'] == 'trig':
          if self.params['hour_enc'] == 'ohe':
            ohe_counts = [(2, 2 + 24)]
          elif self.params['hour_enc'] == 'binned':
            pass
        if self.params['weekday_enc'] == 'binned':
          if self.params['hour_enc'] == 'ohe':
            pass
          elif self.params['hour_enc'] == 'binned':
            pass
          elif self.params['hour_enc'] == 'trig':
            pass
        self.ohe_counts = ohe_counts
        self.valid = []
        if len(ohe_counts) > 0:
          feature_powers = self.poly.powers_
          for i in range(len(feature_powers)):
            # if (feature_powers[i, ohe_counts[0][0]:ohe_counts[0][1]].sum() <= 1
            #     and feature_powers[i, ohe_counts[1][0]:ohe_counts[1][1]].sum() <= 1): self.valid.append(i)
            isValid = True
            for item in ohe_counts:
              if feature_powers[i, item[0]:item[1]].sum() > 1: isValid = False
            if isValid: self.valid.append(i)
      
      if len(self.ohe_counts) > 0:
        X = X[:, self.valid]
    return (X, y)      

  # def _extract_weekday(self, date_string: str) -> int:
  #   return datetime.strptime(date_string, "%Y-%m-%d").weekday()

  def fit(self, df):
    """Constructs the pipeline and trains it."""
    
    X, y = self._process_dataframe(df)

    # --- Split ---
    if self.params['split_rate'] == 0: 
      # This means we don't want to split our data, we want the model to be trained on the whole dataframe.
      # ATTENTION: You're allowed to use self.evaluate only when you've set the split_rate a positive number.
      X_train = X
      y_train = y
    else:
      # This means we want a test subset to be extracted from our dataframe
      X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=self.params['split_rate'], random_state=self.params['random_state'])
      y_test = y_test.values.squeeze()
      self.X_test = X_test
      self.y_test = y_test
    y_train = y_train.values.squeeze()
    self.X_train = X_train
    self.y_train = y_train
    
    # --- PipeLine: Feature scaling & Training ---
    if self.params['reg_type'] == 'lasso':
      if self.params['trainer'] == None:
        self.params['trainer'] = LassoCV(cv=5, random_state=self.params['random_state'], n_jobs=-1, max_iter=10000)
      self.pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('lasso', self.params['trainer'])
      ])
    elif self.params['reg_type'] == 'ridge':
      if self.params['trainer'] == None:
        self.params['trainer'] = RidgeCV()
      self.pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('ridge', self.params['trainer'])        
      ])
    elif self.params['reg_type'] == 'regressor':
      if self.params['trainer'] == None:
        self.params['trainer'] = LinearRegression()
      self.pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('regressor', self.params['trainer'])        
      ])
    else: raise Exception("this reg_type is not supported or is not written correctly, please double check. supported reg_type values: lasso, ridge, regressor")
  
    self.pipeline.fit(X_train, y_train)

    return self

  def predict(self, df, return_labels=False):
    """Gets raw dataframe and outputs model's predictions. Dataframe's structure should be exactly like the original dataframe, with the exception that including Appliance column is optional"""
    X, y = self._process_dataframe(df)
    y_pred = self.pipeline.predict(X)
    if return_labels: return (y_pred, y)
    return y_pred
    
  
  # def evaluate(self, X_test, y_test):
    # """Calculates metrics and stores them in self.results."""
    # y_pred = self.model.predict(X_test)
    # self.results['rmse'] = np.sqrt(mean_squared_error(y_test, y_pred))
    # self.results['r2'] = r2_score(y_test, y_pred)
    # return self.results

## Training Models

### How to Use The Engine: Training a Basic Linear Regression Model

In [4]:
# linear_model = ModelCreator(hour_enc='ohe', weekday_enc='ohe', poly_deg=1, reg_type='regressor', random_state=42
#                             , trainer=LinearRegression(random_state_24)) # Declare specifications
# linear_model.fit(dataframe) # Train the model
# y_pred = linear_model.pipeline.predict(linear_model.X_test) # Raise answers
# RMSE = np.sqrt(mean_squared_error(linear_model.y_test, y_pred)) # calculate error
# RMSE

### Choosing The Proper Encoding For Linear(degree=1) Models

Four models were trained differing only in encoding methods. As expected, the model with one-hot encoding for both variables wins in generalization(due to ratios being closer to one), it also has the highest ability of explaining the variance of the data(Highest $R^2$) along with lowest average prediction inaccuracy(lowest RMSE on both train and test). However, it's obvious that all these models are underfitting and giving us garbage results, we can see that by comaring MAE(represents average distance between the model's predicted value and actual value) **MAE=50.88(Wh)** to average(mean) energy consumption in our data **mean=97.69(Wh)**. It's clear that the model is so innaccurate in guessing our target variable.

| Deg | Hour | Week | Reg | Train R² | Test R² | Train RMSE | Test RMSE | Ratio (Test/Train RMSE) | Test MAE |
|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
| 1 | ohe | ohe | lasso | 0.235 | 0.232 | 90.19 | 87.68 | 0.972 | 50.88 |
| 1 | ohe | trig | lasso | 0.232 | 0.228 | 90.37 | 87.92 | 0.973 | 50.85 |
| 1 | trig | ohe | lasso | 0.203 | 0.205 | 92.08 | 89.19 | 0.969 | 51.68 |
| 1 | trig | trig | lasso | 0.200 | 0.201 | 92.26 | 89.40 | 0.969 | 51.62 |

In [ ]:
# Define your experimental grid
experiments = [
    {'poly_deg': 1, 'hour_enc': 'ohe', 'weekday_enc': 'ohe', 'reg_type': 'lasso', 'trainer': LassoCV(cv=5, random_state=24, n_jobs=-1, max_iter=10000)},
    {'poly_deg': 1, 'hour_enc': 'ohe', 'weekday_enc': 'trig', 'reg_type': 'lasso', 'trainer': LassoCV(cv=5, random_state=24, n_jobs=-1, max_iter=10000)},
    {'poly_deg': 1, 'hour_enc': 'trig', 'weekday_enc': 'ohe', 'reg_type': 'lasso', 'trainer': LassoCV(cv=5, random_state=24, n_jobs=-1, max_iter=10000)},
    {'poly_deg': 1, 'hour_enc': 'trig', 'weekday_enc': 'trig', 'reg_type': 'lasso', 'trainer': LassoCV(cv=5, random_state=24, n_jobs=-1, max_iter=10000)},
]

print("| Deg | Hour | Week | Reg | Train R² | Test R² | Train RMSE | Test RMSE | Ratio (Test/Train RMSE) | Test MAE |")
print("|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|")

for params in experiments:
    # 1. Initialize and Fit
    model = ModelCreator(**params, random_state=42)
    model.fit(dataframe) # Assuming your raw data is in 'dataframe'
    
    # 2. Get Metrics
    # (Using the evaluate method logic we discussed)
    y_train_pred = model.pipeline.predict(model.X_train)
    y_test_pred = model.pipeline.predict(model.X_test)
    
    tr_r2 = r2_score(model.y_train, y_train_pred)
    ts_r2 = r2_score(model.y_test, y_test_pred)
    tr_rmse = np.sqrt(mean_squared_error(model.y_train, y_train_pred))
    ts_rmse = np.sqrt(mean_squared_error(model.y_test, y_test_pred))
    
    mae = mean_absolute_error(model.y_test, y_test_pred)
    
    rmse_ratio = ts_rmse / tr_rmse
    
    # 3. Print Markdown Row
    print(f"| {params['poly_deg']} | {params['hour_enc']} | {params['weekday_enc']} | {params['reg_type']} | "
          f"{tr_r2:.3f} | {ts_r2:.3f} | {tr_rmse:.2f} | {ts_rmse:.2f} | {rmse_ratio:.3f} | {mae:.2f} |")

| Deg | Hour | Week | Reg | Train R² | Test R² | Train RMSE | Test RMSE | Ratio (Test/Train RMSE) | Test MAE |
|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
| 1 | ohe | ohe | lasso | 0.235 | 0.232 | 90.19 | 87.68 | 0.972 | 50.88 |
| 1 | ohe | trig | lasso | 0.232 | 0.228 | 90.37 | 87.92 | 0.973 | 50.85 |
| 1 | trig | ohe | lasso | 0.203 | 0.205 | 92.08 | 89.19 | 0.969 | 51.68 |
| 1 | trig | trig | lasso | 0.200 | 0.201 | 92.26 | 89.40 | 0.969 | 51.62 |


### Choosing The Proper Regularization Type For Linear(degree=1) Models
We concluded from the previous cell that OHE encoding is the best for our linear models, three models with different regularization methods were tested and they all gave the same results. The reason to this is obvious, we talked earlier briefly in the ReadMe that $\lambda$ tries to move the model in bias-variance tradeoff towards the sweetspot. Our linear model is underfitting, it has high bias and low variance, therefore the model tries to increase the model's variance to move it towards the sweetspot by decreasing $\lambda$, but it reaches the model's boundries, meaning $\lambda$ gets near zero and tries it best to increase the variance, but the model has reached its limits and it can't go further(can't have more variance). Hence, all models, regardless of their regularization method, reach the same spot.

In another words, even when $\lambda = 0$(normal regression without regularization) the model is underfitting, meaning it has high bias and what it needs is more variance, and any $\lambda > 0$ provides less variance, hence, the model tries to push $\lambda$ towards zero, pushing all regulrazied regressions towards becoming a normal regression without regularization, therefore all four models end up being the same(not excatly the same, but almost identical).

Considering the observations above, we conclude that when our model is underfitting, there's nothing regularization L1 or L2 can do, but to give us the same model we had. Utilizing regularization methods like L1 and L2 when the model is underfitting doesn't provide any value.

Regularization is meant to reduce model's complexity to decrease variance, using it when the model is too simple and needs more variance doesn't help, therefore, the best linear model with degree one is a regular linear regression model with one-hot encoded weekdays and hours but without regularization.

| Deg | Hour | Week | Reg | Train R² | Test R² | Train RMSE | Test RMSE | Ratio (Test/Train RMSE) | Best $\lambda$ | CrossVal |
|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
| 1 | ohe | ohe | lasso | 0.235 | 0.232 | 90.19 | 87.68 | 0.972 | 0.0197 | 5 Folds |
| 1 | ohe | ohe | ridge | 0.235 | 0.232 | 90.19 | 87.68 | 0.972 | 11.4976 | LOOCV |
| 1 | ohe | ohe | ridge | 0.235 | 0.232 | 90.19 | 87.68 | 0.972 | 11.4976 | 5 Folds |
| 1 | ohe | ohe | regressor | 0.235 | 0.232 | 90.19 | 87.69 | 0.972 | - | - |

But you might ask, if L1 and L2 try to push the $\lambda$ to zero for this model, why does this happens for lasso($\lambda = 0.0197$) but not for ridge($\lambda = 11.4976$)? Even when we try to train a ridge model with alphas=[0.0197, 11.4976], we expect the model to pick the less lambda based on the logic we provided above to increase the model's variance, but it picks 11.4976(check the code below to see), does this mean our logic is wrong? Well, not percisely.

While it seems that ridge is working against this logic, this is not actually true. I trained two ridge models, one with $\lambda = 0.0197$ and another with $\lambda = 11.4976$, and look these wonderful table, they both raised the same results! This means both $\lambda$ values give our model almost the same variance, they both create the same model. But why?

The reason comes from how Ridge treats our data. Lasso is much more aggressive in penalizing variance comparing to Ridge, hence it's much more sensitive to changing the value of $\lambda$. As shown in the table below, the same change for $\lambda$ effects Lasso regression remarkably while Ridge regression stays almost the same.

But if ridge regression with $\lambda = 0.0197$ and $\lambda = 11.4976$ are almost identical, why doesn't the model choose the lower lambda? that lower lambda might have the same performance but just be a little bit better, but this amount is so small the model can't see it properly.

Our data is noisy, and K-Fold cross validation isn't perfect. Lasso with $\lambda = 0.0197$ might be $0.00001$ better in performance, but the noise and very little uncontrolable bias that can happen in K-Fold CV, can hide this fact, making the model think $\lambda = 11.4976$ is an option that's a little bit better.

| Deg | Hour | Week | Reg | Train R² | Test R² | Train RMSE | Test RMSE | Ratio (Test/Train RMSE) | Fixed $\lambda$ | CrossVal |
|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
| 1 | ohe | ohe | ridge | 0.235 | 0.232 | 90.19 | 87.69 | 0.972 | 0.0197 | 5 Folds |
| 1 | ohe | ohe | ridge | 0.235 | 0.232 | 90.19 | 87.68 | 0.972 | 11.4976 | 5 Folds |
| 1 | ohe | ohe | lasso | 0.235 | 0.232 | 90.19 | 87.68 | 0.972 | 0.0197 | 5 Folds |
| 1 | ohe | ohe | lasso | 0.056 | 0.060 | 100.22 | 97.01 | 0.968 | 11.4976 | 5 Folds |

Here's a visual summary:

<img src="./visuals/diagram-bias-variance-tradeoff.png" alt="bias variance tradeoff explained with diagram" >

In [6]:
# Define your experimental grid
alphas = np.logspace(-3, 3, 100)
experiments = [
    {'poly_deg': 1, 'hour_enc': 'ohe', 'weekday_enc': 'ohe', 'reg_type': 'lasso', 'trainer': LassoCV(cv=5, random_state=24, n_jobs=-1, max_iter=10000)},
    {'poly_deg': 1, 'hour_enc': 'ohe', 'weekday_enc': 'ohe', 'reg_type': 'ridge', 'trainer': RidgeCV(cv=None, alphas=alphas)},
    {'poly_deg': 1, 'hour_enc': 'ohe', 'weekday_enc': 'ohe', 'reg_type': 'ridge', 'trainer': RidgeCV(cv=5, alphas=alphas)},
    {'poly_deg': 1, 'hour_enc': 'ohe', 'weekday_enc': 'ohe', 'reg_type': 'regressor', 'trainer': LinearRegression()},
    {'poly_deg': 1, 'hour_enc': 'ohe', 'weekday_enc': 'ohe', 'reg_type': 'ridge', 'trainer': RidgeCV(cv=5, alphas=[0.0197, 11.4976])},
    {'poly_deg': 1, 'hour_enc': 'ohe', 'weekday_enc': 'ohe', 'reg_type': 'ridge', 'trainer': RidgeCV(cv=5, alphas=[0.0197])},
    {'poly_deg': 1, 'hour_enc': 'ohe', 'weekday_enc': 'ohe', 'reg_type': 'ridge', 'trainer': RidgeCV(cv=5, alphas=[11.4976])},
    {'poly_deg': 1, 'hour_enc': 'ohe', 'weekday_enc': 'ohe', 'reg_type': 'lasso', 'trainer': LassoCV(cv=5, alphas=[0.0197], random_state=24, n_jobs=-1, max_iter=10000)},
    {'poly_deg': 1, 'hour_enc': 'ohe', 'weekday_enc': 'ohe', 'reg_type': 'lasso', 'trainer': LassoCV(cv=5, alphas=[11.4976], random_state=24, n_jobs=-1, max_iter=10000)},
]

print("| Deg | Hour | Week | Reg | Train R² | Test R² | Train RMSE | Test RMSE | Ratio (Test/Train RMSE) | Best $\\lambda$ |")
print("|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|")

for params in experiments:
    # 1. Initialize and Fit
    model = ModelCreator(**params, random_state=42)
    model.fit(dataframe) # Assuming your raw data is in 'dataframe'
    
    # 2. Get Metrics
    # (Using the evaluate method logic we discussed)
    y_train_pred = model.pipeline.predict(model.X_train)
    y_test_pred = model.pipeline.predict(model.X_test)
    
    tr_r2 = r2_score(model.y_train, y_train_pred)
    ts_r2 = r2_score(model.y_test, y_test_pred)
    tr_rmse = np.sqrt(mean_squared_error(model.y_train, y_train_pred))
    ts_rmse = np.sqrt(mean_squared_error(model.y_test, y_test_pred))
    
    rmse_ratio = ts_rmse / tr_rmse
    
    try:
        alpha = model.pipeline.named_steps[params['reg_type']].alpha_
    except AttributeError: alpha = -1
    
    # 3. Print Markdown Row
    print(f"| {params['poly_deg']} | {params['hour_enc']} | {params['weekday_enc']} | {params['reg_type']} | "
          f"{tr_r2:.3f} | {ts_r2:.3f} | {tr_rmse:.2f} | {ts_rmse:.2f} | {rmse_ratio:.3f} | {alpha:.4f} |")

| Deg | Hour | Week | Reg | Train R² | Test R² | Train RMSE | Test RMSE | Ratio (Test/Train RMSE) | Best $\lambda$ |
|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
| 1 | ohe | ohe | lasso | 0.235 | 0.232 | 90.19 | 87.68 | 0.972 | 0.0197 |
| 1 | ohe | ohe | ridge | 0.235 | 0.232 | 90.19 | 87.68 | 0.972 | 11.4976 |
| 1 | ohe | ohe | ridge | 0.235 | 0.232 | 90.19 | 87.68 | 0.972 | 11.4976 |
| 1 | ohe | ohe | regressor | 0.235 | 0.232 | 90.19 | 87.69 | 0.972 | -1.0000 |
| 1 | ohe | ohe | ridge | 0.235 | 0.232 | 90.19 | 87.68 | 0.972 | 11.4976 |
| 1 | ohe | ohe | ridge | 0.235 | 0.232 | 90.19 | 87.69 | 0.972 | 0.0197 |
| 1 | ohe | ohe | ridge | 0.235 | 0.232 | 90.19 | 87.68 | 0.972 | 11.4976 |
| 1 | ohe | ohe | lasso | 0.235 | 0.232 | 90.19 | 87.68 | 0.972 | 0.0197 |
| 1 | ohe | ohe | lasso | 0.056 | 0.060 | 100.22 | 97.01 | 0.968 | 11.4976 |


### Choosing The Best Polynomial Model(degree two)

As we saw, linear models with degree one could not explain our data prorperly. They were underfitting, lacking variance. Therefore, we increase the complexity of our model by changing its polynomial degree to two, to get a model with higher variance that has more chance to learn our data, due to its flexiblity to learn new patterns and less bias on the patterns it already knows.

#### Tuning Problem: LassoCV
When training a lasso regression with one-hot encoded columns, I got a 'ConvergenceWarning' from scikit-learn. The data had over 1400 features with high multicollinearity due to a lot of columns representing same information or representing information that can be extraced from other columns. These multicollinearities happen often with noisy datasets like the one we are working with right now.

Becuase the data had a lot of redunant features, the loss function plot's shape had become nearly flat around the optimal point. Lasso takes steps in a path towards that optimal point, and when it gets close, the shape becomes flat meaning lasso will take smaller and smaller steps, when we get to a point that we have taken lots of steps but we haven't reached the optimal point yet. This was the reason why we got the 'ConvergenceWarning' warning.

To fix this problem, the hyperparmeters tol and max_iter needed to be tuned properly. max_iter is the maximum number of steps our lasso model takes to reach the optimal point. And to know about tol, first we have to know what is $Duality Gap$.

Computitions aren't perfect. Computer can not always raech the exact optimal spot, but it can get very close to it, by taking steps in a path towards the point and stopping somewhere near the optimal point. To determine when we should stop, we define duality gap. Duality gap is the difference of the theoretical possible minimum for our cost function and the current value of our cost function. The theoretical minimums is calculated using mathematic relations. We can think of duality gap as the distance is left to reach the optimal point. Hence, we define tol. We tell the model that if your distance from the optimal point(duality gap) is less than the value of tol, you are close enough(convergence has happened), you are good to stop computitions and consider the weights of the current spot as an answer.

$$\text{Duality Gap} = \text{Current Cost (Primal)} - \text{Theoretical Floor (Dual)}$$

The problem of our model was that it would stop in a spot that was not close enough to the optimal spot($Duality Gap > tol$) because it had reached the maximum number of steps it could take(max_iter). This means that convergence didn't happen, resulting in a 'ConvergenceWarning'. The model had finished the training and all the process without error, but its answer is not accurate and 'good enough'. Therefore, by increasing the value of tol and max_iter, we can make sure convergunce happens.

The cost function used in LassoCV is:

$$J(w) = \frac{1}{2n_{\text{samples}}} \|y - Xw\|_2^2 + \alpha \|w\|_1$$

The second term is L1 norm, the first term is actually $\frac{1}{2} MSE$. It's clear that $2 \times J(w) > MSE$ and $J_{\text{answer}}(w) - J_{\text{theoretical minimum}}(w) < tol$ and $MAE < MSE$. This means the difference of our final $MAE$ and the smallest possible $MAE$ is guaranteed to be smaller than two times the value of our $tol$. This logic helps us to increase $tol$ with open eyes without making the model too much innaccurate.

$MAE$ shows the average innaccuracy of our model in predicting the target variable. Our target variable has an average(mean) value of 50(Wh). Hence, if the average error of our model is 0.05(Wh) more than the best $MAE$, it's still good, hence $tol=0.1$ seems like a good number. The previous $tol=10^{-4}$ was an overkill for this dataset.

This increase in $tol$ means the model can reach a spot with more distance from the optimal value than before, but still be considered as a convergunce. This makes the destination more accessible, increasing the probablity that convergunce happens in a fair amount of steps. But still, we have to increase max_iter to let the model be able to take more steps, again increasing convergunce probablity. $40000$ seems like a good number.

In [17]:
def print_metrics(model):
    y_train_pred = model.pipeline.predict(model.X_train)
    y_test_pred = model.pipeline.predict(model.X_test)
    
    tr_r2 = r2_score(model.y_train, y_train_pred)
    ts_r2 = r2_score(model.y_test, y_test_pred)
    tr_rmse = np.sqrt(mean_squared_error(model.y_train, y_train_pred))
    ts_rmse = np.sqrt(mean_squared_error(model.y_test, y_test_pred))
    
    rmse_ratio = ts_rmse / tr_rmse
    r2_ratio = ts_r2 / tr_r2
    
    try:
        alpha = model.pipeline.named_steps[params['reg_type']].alpha_
    except AttributeError: alpha = -1    
    
    print(f"| {params['poly_deg']} | {params['hour_enc']} | {params['weekday_enc']} | {params['reg_type']} | "
          f"{tr_r2:.3f} | {ts_r2:.3f} | {tr_rmse:.2f} | {ts_rmse:.2f} | {rmse_ratio:.3f} | {r2_ratio:.3f} | {alpha:.4f} |")

In [ ]:
# Define your experimental grid
experiments = [
    {'poly_deg': 2, 'hour_enc': 'ohe', 'weekday_enc': 'ohe', 'reg_type': 'lasso', 'trainer': LassoCV(cv=5, random_state=24, tol=0.1, n_jobs=-1, max_iter=40000)},
]
lasso_ohe_ohe = ModelCreator(**experiments[0], random_state=42)
lasso_ohe_ohe.fit(dataframe)

| Deg | Hour | Week | Reg | Train R² | Test R² | Train RMSE | Test RMSE | Ratio (Test/Train RMSE) |
|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|


In [ ]:
print("| Deg | Hour | Week | Reg | Train R² | Test R² | Train RMSE | Test RMSE | Ratio (Test/Train RMSE) | Ratio($R^2$) | best $\\lambda$ |")
print("|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|")
print_metrics(lasso_ohe_ohe)

| Deg | Hour | Week | Reg | Train R² | Test R² | Train RMSE | Test RMSE | Ratio (Test/Train RMSE) | Ratio($R^2$) | best $\lambda$ |
|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|


| Deg | Hour | Week | Reg | Train R² | Test R² | Train RMSE | Test RMSE | Ratio (Test/Train RMSE) |
|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
| 2 | ohe | ohe | lasso | 0.392 | 0.333 | 80.45 | 81.70 | 1.016 |

In [18]:
experiments = [
    {'poly_deg': 2, 'hour_enc': 'ohe', 'weekday_enc': 'ohe', 'reg_type': 'ridge', 'trainer': RidgeCV(cv=None, alphas=alphas)},
    {'poly_deg': 2, 'hour_enc': 'ohe', 'weekday_enc': 'trig', 'reg_type': 'ridge', 'trainer': RidgeCV(cv=None, alphas=alphas)},
    {'poly_deg': 2, 'hour_enc': 'trig', 'weekday_enc': 'ohe', 'reg_type': 'ridge', 'trainer': RidgeCV(cv=None, alphas=alphas)},
    {'poly_deg': 2, 'hour_enc': 'trig', 'weekday_enc': 'trig', 'reg_type': 'ridge', 'trainer': RidgeCV(cv=None, alphas=alphas)}
]

for params in experiments:
    model = ModelCreator(**params, random_state=42)
    model.fit(dataframe)
    print_metrics(model)

| 2 | ohe | ohe | ridge | 0.473 | 0.347 | 74.86 | 80.84 | 1.080 | 0.733 | 0.0071 |
| 2 | ohe | trig | ridge | 0.445 | 0.343 | 76.83 | 81.08 | 1.055 | 0.771 | 0.0040 |
| 2 | trig | ohe | ridge | 0.403 | 0.331 | 79.69 | 81.80 | 1.027 | 0.822 | 0.0035 |
| 2 | trig | trig | ridge | 0.374 | 0.324 | 81.60 | 82.24 | 1.008 | 0.867 | 0.0023 |


| Deg | Hour | Week | Reg | Train R² | Test R² | Train RMSE | Test RMSE | Ratio (Test/Train RMSE) | Ratio($R^2$) | best $\lambda$ |
|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
| 2 | ohe | ohe | ridge | 0.473 | 0.347 | 74.86 | 80.84 | 1.080 | 0.733 | 0.0071 |
| 2 | ohe | trig | ridge | 0.445 | 0.343 | 76.83 | 81.08 | 1.055 | 0.771 | 0.0040 |
| 2 | trig | ohe | ridge | 0.403 | 0.331 | 79.69 | 81.80 | 1.027 | 0.822 | 0.0035 |
| 2 | trig | trig | ridge | 0.374 | 0.324 | 81.60 | 82.24 | 1.008 | 0.867 | 0.0023 |

In [19]:
experiments = [
    {'poly_deg': 2, 'hour_enc': 'ohe', 'weekday_enc': 'ohe', 'reg_type': 'ridge', 'trainer': RidgeCV(cv=5, alphas=alphas)},
    {'poly_deg': 2, 'hour_enc': 'ohe', 'weekday_enc': 'trig', 'reg_type': 'ridge', 'trainer': RidgeCV(cv=5, alphas=alphas)},
    {'poly_deg': 2, 'hour_enc': 'trig', 'weekday_enc': 'ohe', 'reg_type': 'ridge', 'trainer': RidgeCV(cv=5, alphas=alphas)},
    {'poly_deg': 2, 'hour_enc': 'trig', 'weekday_enc': 'trig', 'reg_type': 'ridge', 'trainer': RidgeCV(cv=5, alphas=alphas)}
]

for params in experiments:
    model = ModelCreator(**params, random_state=42)
    model.fit(dataframe)
    print_metrics(model)

KeyboardInterrupt: 